In [20]:
#importing the required modules
import os, json, csv
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk
import numpy as np
from datetime import datetime, timedelta

In [21]:
dataset_path = "plates"  #path containing the Dubai number plates
img_height = 224 
img_width = 224
batch_size = 32  #number of images processing in one training batch
epochs = 15  #training iterations
model_path = "trained_dubai_model.keras"    #poth to save the trained model
class_names_path = "dubai_class_names.json"   #store the class labels

In [22]:
#loading dataset as 80% for training
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,    #20% reserved for testing
    subset="training",
    seed=123,    #consistent date shuffling
    image_size=(img_height, img_width),
    batch_size=batch_size
)

#loading dataset as 20% for testing
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Dubai Plate Classes:", class_names)

Found 348 files belonging to 10 classes.
Using 279 files for training.
Found 348 files belonging to 10 classes.
Using 69 files for validation.
Dubai Plate Classes: ['C5038', 'D77988', 'H79111', 'I9269', 'Q5121', 'R6729', 'S3227', 'S3601', 'U8804', 'X89099']


In [23]:
#performance optimization
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

#augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.2),
])

In [24]:
#CNN model
model = models.Sequential([
    Input(shape=(img_height, img_width, 3)),
    data_augmentation, 
    layers.Rescaling(1./255),

    #convolutional features
    layers.Conv2D(32, 3, activation='relu'),  #first convolunational layer with 32 filters
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation='relu'),  #second convolutional layer with 64 filters
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation='relu'),  #third convolutional layer with 128 filters
    layers.MaxPooling2D(),

    layers.Flatten(), #flattening the feature maps into a single vector
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax') #output layer for multi-class classification
])

#compiling the model with an Adam operator
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#printing model architecture summary
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)            │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling_2 (Rescaling)              │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,170,250 (42.61 MB)

 Trainable params: 11,170,250 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
history = model.fit(
    train_ds,  #training dataset
    validation_data=val_ds, #testing dataset
    epochs=15 #training epochs
)

model.save("trained_dubai_model.keras")

with open("class_names.json", "w") as f:
    json.dump(class_names, f)

print("Model and class names saved.")

Epoch 1/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 14s 955ms/step - accuracy: 0.1107 - loss: 3.1665 - val_accuracy: 0.1304 - val_loss: 2.0867
Epoch 2/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 867ms/step - accuracy: 0.1524 - loss: 2.0324 - val_accuracy: 0.1304 - val_loss: 1.9434
Epoch 3/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 886ms/step - accuracy: 0.2440 - loss: 1.9105 - val_accuracy: 0.3913 - val_loss: 1.7628
Epoch 4/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 997ms/step - accuracy: 0.3549 - loss: 1.7281 - val_accuracy: 0.6957 - val_loss: 1.2842
Epoch 5/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6348 - loss: 1.2330 - val_accuracy: 0.8696 - val_loss: 0.4858
Epoch 6/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8066 - loss: 0.6418 - val_accuracy: 1.0000 - val_loss: 0.2627
Epoch 7/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9476 - loss: 0.3117 - val_accuracy: 1.0000 - val_loss: 0.0647
Epoch 8/15
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9693 - loss: 0.0709 - val_accuracy: 1.0000 - val_loss: 0.0016